In [ ]:
%pylab inline

In [ ]:
from tmqr.settings import *
from tmqrfeed import DataManager
from pymongo import MongoClient
from tmqrindex.index_base import IndexBase
from datetime import datetime
import re
import pandas as pd

In [ ]:
client = MongoClient(MONGO_CONNSTR)
db = client[MONGO_DB]

dm = DataManager()

In [ ]:
def get_exo_list(exo_filter='*', return_names=True):
    """
    Return EXO list stored in MongoDB V2
    :param exo_filter: '*' - include all, wildcard is allowed (like, 'ES_Bullish*')
    :param return_names: if True returns names list of EXO, otherwize returns MongoDB data collection list
    :return: list of EXO names
    """
    re_val = exo_filter.replace('*','.*')

    data = db['index_data'].find({'name': re.compile(re_val, re.IGNORECASE)})
    if return_names:
        return [exo['name'] for exo in data]
    else:
        return list(data)

In [ ]:
#exo_filter = 'neutralOnly'     # All 
exo_filter = '*'  # ES only
#exo_filter = '*'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

exo_dict = {}
for exo in get_exo_list(exo_filter, return_names=False):
    idx = IndexBase.deserialize(dm, exo, as_readonly=True)
    exo_dict[idx.index_name] = idx

In [ ]:
for exo_name, exo in exo_dict.items():
    exo_df = exo.data
    
    if len(exo_df) < 200:
        print("{0:<70} [NODATA DataLen: {1}]".format(exo.index_name, len(exo_df)))
    elif (datetime.now().date() - exo_df.index.date[-1]).days > 4:
        print("{0:<70} [DELAYED: LastDate: {1}]".format(exo.index_name, exo_df.index[-1]))
    else:
        print("{0:<70} [OK]".format(exo.index_name))

In [ ]:
#exo_dict['US.ES_ContFutEOD'].data

# Single EXO view

In [ ]:
EXO_NAME = 'US.CL_EXOSemiFuture_Delta20'

In [ ]:
idx = exo_dict[EXO_NAME]

In [ ]:
exo_df = idx.data
figsize(8,10)
if 'c' in exo_df:
    # This is Quote* algorithm based index
    exo_df['c'].plot(title=f'EXO (Close price): {EXO_NAME}');
else:    
    f, (ax1, ax2, ax3) = plt.subplots(3, gridspec_kw = {'height_ratios':[3, 1, 1]})
    exo_df['equity_decision'].plot(ax=ax1);
    ax1.set_title('EXO: {0}'.format(EXO_NAME));

    exo_df['costs'].cumsum().plot(ax=ax2);
    ax2.set_title('Cumulative costs');
    ax2.set_xticklabels([]);

    (exo_df['ncontracts_executed']+exo_df['noptions_executed']).plot(ax=ax3)
    ax3.set_title("#Contracts executed");
    
plt.tight_layout()

## EXO Statistics

In [ ]:
stdev_monthly = exo_df['equity_decision'].diff(22).std()
stdev_daily = exo_df['equity_decision'].diff(1).std()
nyears = (exo_df.index[-1] - exo_df.index[0]).days / 365.25
contracts_executed_per_year = (exo_df['ncontracts_executed'].sum() + exo_df['noptions_executed'].sum()) / nyears

print("EXO statistics")
print("{0:<20} {1:>10.1f}".format('Daily StDev', stdev_daily))
print("{0:<20} {1:>10.1f}".format('Monthly StDev', stdev_monthly))
print("{0:<20} {1:>10.1f}".format('Contracts exec./ yr.', contracts_executed_per_year))

In [ ]:
figsize(10, 5)
f, (ax1, ax2) = plt.subplots(2)

diff1 = exo_df['equity_decision'].diff(1).dropna()
diff20 = exo_df['equity_decision'].diff(20).dropna()

diff1.dropna().hist(bins=50, ax=ax1);
ax1.set_title('1-day changes histogram');


diff20.dropna().hist(bins=50, ax=ax2);
ax2.set_title('20-day changes histogram');
plt.tight_layout()

In [ ]:
exo_df